In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F


In [2]:
spark = SparkSession.builder.appName("solution_ex_18").getOrCreate()

22/06/03 15:35:13 WARN Utils: Your hostname, certusr-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/06/03 15:35:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/03 15:35:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/03 15:35:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/06/03 15:35:21 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/06/03 15:35:21 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
inputPath = './data'
outputPath = './output_ex_18/df_sol'

print(os.path.exists(inputPath))
print(os.path.exists(outputPath))

True
False


In [10]:
# Create a DataFrame from persons.csv
dfPersons = spark.read.load(inputPath,\
                     format="csv",\
                     header=True,\
                     inferSchema=True)
dfPersons.show()

+-----+---+------+
| name|age|gender|
+-----+---+------+
| Paul| 40|  male|
| Paul| 38|  male|
|Susan| 40|female|
|Karen| 34|female|
| John| 40|  male|
|Susan| 34|female|
+-----+---+------+



In [11]:
gruped_DF = dfPersons.groupby("name").agg({"name":"count", "age":"avg"})
gruped_DF.show()

+-----+-----------+--------+
| name|count(name)|avg(age)|
+-----+-----------+--------+
|Susan|          2|    37.0|
| John|          1|    40.0|
| Paul|          2|    39.0|
|Karen|          1|    34.0|
+-----+-----------+--------+



In [12]:
names_2Times_DF = gruped_DF.filter("count(name) >= 2")
names_2Times_DF.show()

+-----+-----------+--------+
| name|count(name)|avg(age)|
+-----+-----------+--------+
|Susan|          2|    37.0|
| Paul|          2|    39.0|
+-----+-----------+--------+



In [14]:
# Select only the name and average(age) of the selected names
dfSelectedNamesAvgAge = names_2Times_DF.select("name", "avg(age)")
dfSelectedNamesAvgAge.show()

+-----+--------+
| name|avg(age)|
+-----+--------+
|Susan|    37.0|
| Paul|    39.0|
+-----+--------+



In [15]:
dfSelectedNamesAvgAge.write.csv(outputPath, header=False)